In [2]:
# необходимые импорты
import numpy as np
import torch
import torchvision.datasets as dataset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, WeightedRandomSampler
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torchvision import models

import os
import cv2
import onnxruntime
import torch.onnx as onnx

In [13]:
train_path = r"E:\Datasets\Glaucoma_detection\Fundus_Train_Val_Data\Fundus_Scanes_Sorted_And_Cropped_v2\Train"
test_path  = r"E:\Datasets\Glaucoma_detection\Fundus_Train_Val_Data\Fundus_Scanes_Sorted_And_Cropped_v2\Validation"

In [14]:
# определим объект трансформации изображений
no_aug_transform = transforms.Compose(
      [
            transforms.Resize(size=(224, 224)),
            transforms.ToTensor(),
      ]
)

In [5]:
test_data = dataset.ImageFolder(test_path, no_aug_transform)

print(type(test_data))

print(test_data.classes)

<class 'torchvision.datasets.folder.ImageFolder'>
['Glaucoma_Negative', 'Glaucoma_Positive']


In [6]:
print(test_data[0][0])

tensor([[[0.8784, 0.8706, 0.8667,  ..., 0.8667, 0.8745, 0.8706],
         [0.8784, 0.8745, 0.8745,  ..., 0.8667, 0.8745, 0.8784],
         [0.8784, 0.8745, 0.8706,  ..., 0.8667, 0.8745, 0.8745],
         ...,
         [0.8353, 0.8235, 0.8157,  ..., 0.8863, 0.8824, 0.8784],
         [0.8314, 0.8196, 0.8196,  ..., 0.8902, 0.8863, 0.8824],
         [0.8314, 0.8275, 0.8275,  ..., 0.8941, 0.8902, 0.8902]],

        [[0.5373, 0.5333, 0.5373,  ..., 0.5569, 0.5647, 0.5608],
         [0.5412, 0.5412, 0.5451,  ..., 0.5569, 0.5647, 0.5686],
         [0.5412, 0.5412, 0.5451,  ..., 0.5569, 0.5647, 0.5647],
         ...,
         [0.4863, 0.4745, 0.4627,  ..., 0.5373, 0.5255, 0.5216],
         [0.4824, 0.4706, 0.4667,  ..., 0.5373, 0.5294, 0.5255],
         [0.4824, 0.4745, 0.4745,  ..., 0.5373, 0.5333, 0.5333]],

        [[0.1961, 0.1922, 0.2000,  ..., 0.1922, 0.2000, 0.1961],
         [0.2000, 0.1961, 0.2078,  ..., 0.1922, 0.2000, 0.2039],
         [0.2000, 0.1961, 0.2078,  ..., 0.1922, 0.2000, 0.

In [7]:
# определим устройство, на котором будет идти обучение
device = None
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [9]:
state_dict = torch.load(r"..\..\src\weights\classification\pytorch\efficientnetb0_best_checkpoint_crop_optic_disc.pth", map_location=device).state_dict()
inference_model = models.efficientnet_b0(num_classes=2, pretrained=False)
inference_model.load_state_dict(state_dict)
inference_model.eval()
inference_model.to(device)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [10]:
# создаем входной тензор модели
x = torch.randn(1, 3, 224, 224, requires_grad=True).to(device)

# сгенерируем выходы модели
out = inference_model(x)

# экспортируем модель в onnx
onnx.export(inference_model, x, r"..\..\src\weights\classification\onnx\efficientnetb0_best_checkpoint_crop_optic_disc.onnx", input_names=["image"], output_names=["output"], )

In [11]:
import onnx

onnx_model = onnx.load( r"..\..\src\weights\classification\onnx\efficientnetb0_best_checkpoint_crop_optic_disc.onnx")
onnx.checker.check_model(onnx_model)

In [12]:
# инициализируем сессию  ONNXRuntime
ort_session = onnxruntime.InferenceSession(r"..\..\src\weights\classification\onnx\efficientnetb0_best_checkpoint_crop_optic_disc.onnx", providers=["CPUExecutionProvider" if device == "cpu" else "CUDAExecutionProvider"])

# определим функцию перевода тензора в numpy-массив
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# определим входы и выходы 
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)
print(ort_outs)
print(out)

# сравним результаты
np.testing.assert_allclose(to_numpy(out[0]), ort_outs[0][0], rtol=2e-02, atol=1e-03)
print("Все хорошо!")

[array([[-0.91505665,  4.8708334 ]], dtype=float32)]
tensor([[-0.9145,  4.8683]], device='cuda:0', grad_fn=<AddmmBackward0>)
Все хорошо!


In [16]:
EFFICIENTNET_INPUT_WIDTH = ort_session.get_inputs()[0].shape[2]
EFFICIENTNET_INPUT_HEIGHT = ort_session.get_inputs()[0].shape[3]
EFFICIENTNET_INPUT_CHANNELS = ort_session.get_inputs()[0].shape[1]

In [ ]:
# mixed cv2 and PIL preprocess
import cv2
from PIL import Image
import shutil

total_images_count = 0
true_classified_images_count = 0

test_listdir = os.listdir(test_path)
for class_name in test_listdir:
    images_listdir = os.listdir(os.path.join(test_path, class_name))
    for i, image in enumerate(images_listdir):
        path_to_image = os.path.join(test_path, class_name, image)
        print(path_to_image)
        image_array_ = cv2.imdecode(np.fromfile(path_to_image, dtype=np.uint8), cv2.IMREAD_UNCHANGED)
        image_array = np.array(Image.fromarray(image_array_).resize((EFFICIENTNET_INPUT_WIDTH, EFFICIENTNET_INPUT_HEIGHT), Image.BILINEAR))
        image_array = image_array.transpose(2, 0, 1)
        image_array = np.expand_dims(image_array, axis=(0))
        outputs = ort_session.run(None, {ort_session.get_inputs()[0].name: image_array.astype("float32")})
        preds_id = outputs[0].argmax()
        class_name_pred = test_data.classes[preds_id]
        if class_name == class_name_pred:
            true_classified_images_count += 1
        else:
            print("\n" + f"label {class_name}, predicted {class_name_pred}")
        total_images_count += 1

E:\Datasets\Glaucoma_detection\Fundus_Train_Val_Data\Fundus_Scanes_Sorted_And_Cropped_v2\Validation\Glaucoma_Negative\483.jpg
E:\Datasets\Glaucoma_detection\Fundus_Train_Val_Data\Fundus_Scanes_Sorted_And_Cropped_v2\Validation\Glaucoma_Negative\484.jpg
E:\Datasets\Glaucoma_detection\Fundus_Train_Val_Data\Fundus_Scanes_Sorted_And_Cropped_v2\Validation\Glaucoma_Negative\485.jpg
E:\Datasets\Glaucoma_detection\Fundus_Train_Val_Data\Fundus_Scanes_Sorted_And_Cropped_v2\Validation\Glaucoma_Negative\487.jpg
E:\Datasets\Glaucoma_detection\Fundus_Train_Val_Data\Fundus_Scanes_Sorted_And_Cropped_v2\Validation\Glaucoma_Negative\488.jpg
E:\Datasets\Glaucoma_detection\Fundus_Train_Val_Data\Fundus_Scanes_Sorted_And_Cropped_v2\Validation\Glaucoma_Negative\489.jpg
E:\Datasets\Glaucoma_detection\Fundus_Train_Val_Data\Fundus_Scanes_Sorted_And_Cropped_v2\Validation\Glaucoma_Negative\491.jpg
E:\Datasets\Glaucoma_detection\Fundus_Train_Val_Data\Fundus_Scanes_Sorted_And_Cropped_v2\Validation\Glaucoma_Negative\

In [24]:
true_classified_images_count / total_images_count

0.7384615384615385